In [1]:
import requests
import asyncio
from playwright.async_api import async_playwright
from urllib.parse import urljoin
from pyquery import PyQuery as pq
import time
import json
import re
from motor.motor_asyncio import AsyncIOMotorClient
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

In [7]:
BASE_URL = 'https://www.qzsc.cc/'
cat_name = {
    'xuanhuanxiaoshuo/': '玄幻',
    'xiuzhenxiaoshuo/': '修真',
    'dushixiaoshuo/': '都市',
    }

In [8]:
for key, value in cat_name.items():
    print(key, value)
    print(type(key), type(value))
    url = urljoin(BASE_URL, key)
    print(url)

xuanhuanxiaoshuo/ 玄幻
<class 'str'> <class 'str'>
https://www.qzsc.cc/xuanhuanxiaoshuo/
xiuzhenxiaoshuo/ 修真
<class 'str'> <class 'str'>
https://www.qzsc.cc/xiuzhenxiaoshuo/
dushixiaoshuo/ 都市
<class 'str'> <class 'str'>
https://www.qzsc.cc/dushixiaoshuo/


In [2]:
string = '作    者：雪月天歌'

In [4]:
re.search(r'者：(.*)', string).group(1)

'雪月天歌'

In [5]:
doc = pq(filename='xs.html')

In [6]:
author = re.search(r'者：(.*)', doc('#info p:contains(者：)').text()).group(1)
last_update = re.search(r'更新：(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})', doc('#info p:contains(更新：)').text()).group(1)
brief = doc('#intro p').text()

In [7]:
print(author)

作    者：笔落尘凡 状    态： 连载, 【加入书架】, 【直达底部】 最后更新：2024-02-05 18:42:45 最新章节： 第581章 ：山河失色！区区小劫！


In [8]:
doc('#info p:contains(者：)').text()

'作\xa0\xa0\xa0\xa0者：笔落尘凡'

In [9]:
re.search(r'者：(.*)', doc('#info p:contains(者：)').text()).group(1)

'笔落尘凡'

In [10]:
re.search(r'更新：(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})', doc('#info p:contains(更新：)').text()).group(1)

'2024-02-05 18:42:45'

In [13]:
brief = doc('#intro p').text()

In [18]:
BASE_URL = 'https://www.qzsc.cc/'

In [19]:
chapters = doc('.box_con #list a')
chapter_info = []
for chapter in chapters.items():
    url = urljoin(BASE_URL, chapter.attr('href'))
    title = chapter.text()
    chapter_info.append({
        'url': url,
        'title': title
    })
print(chapter_info)

https://www.qzsc.cc/qz/153/153591/75736616.html 第一章：古钟三响！洛氏之危！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/90802420.html 第二章：力不从心，自此封山！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/55950037.html 第三章：太虚三章！寻求生机！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/64744929.html 第四章：寻找新的资源点！孤星山脉！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/55237676.html 第五章 ：深入蛮荒！灵脉之心！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/400430.html 第六章：聚四方灵气！灵脉之心融合！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/54572705.html 第七章：炼气九重！灵脉之心！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/30790163.html 第八章：巨大的收获！人心难测！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/90704490.html 第九章：回青山郡，赵氏修士！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/50583298.html 第十章：危机暂缓，筑基丹！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/69915812.html 第十一章：家族任务，阵法之说！
<class 'str'> <class 

In [1]:
book_url = {
    'name': 'feitu',
    'chapter_urls': ['http://www.qzxs.cc/xiaoshuo/rbd2vo1/chapter/g2g5vo1.html', 'http://www.qzxs.cc/xiaoshuo/rbd2vo1/chapter/x2g5vo1.html', 'http://www.qzxs.cc/xiaoshuo/rbd2vo1/chapter/y2g5vo1.html']
}

In [2]:
book_url.get('name')

'feitu'

In [4]:
'''
程序作用: 爬取指定页面，返回页面文本
参数: url (爬取页面地址)
返回值: response.text (页面文本)
'''
def scrape_page(url):   # 定义函数为scrape_page
    # url = 'https://ssr1.scrape.center/page/1' 
    logging.info('scraping %s', url)  # 以info级别的日志信息输出这个“url"网址  
    # xxx(xxx, xxx); xxx(xxx) scraping https://ssr1.scrape.center/page/1
    try: # 异常处理 try 和 except
        response = requests.get(url)    # 爬取 请求网址响应
        if response.status_code == 200: # 状态码为200时返回页面文本 
            return response.text        # 返回页面文本 
        logging.info('get invalid status code %s while scraping %s', response.status_code, url)
        # 以info级别的日志信息输出 “获取无效状态代码  或者 解析失败   url= https://ssr1.scrape.center/page/1”
    except requests.RequestException:    # 如果请求异常 
        logging.error('error occurred while scraping %s', url, exc_info=True) 
        # 以error级别的日志信息输出 “请求异常  url= https://ssr1.scrape.center/page/1”

In [5]:
URL = 'http://www.qzxs.cc/xiaoshuo/rbd2vo1/chapter/32g5vo1.html'

In [6]:
print(scrape_page(URL))

2024-02-17 23:06:02,550 - INFO: scraping http://www.qzxs.cc/xiaoshuo/rbd2vo1/chapter/32g5vo1.html


<!DOCTYPE html><html lang="zh-cmn-Hans" class="theme5"><head><meta charset="UTF-8"><title>第四百六七节 幽灵-废土(黑天魔神)-棋子小说</title><meta name="keywords" content="第四百六七节 幽灵,废土,黑天魔神,棋子小说"><meta name="description" content="第四百六七节 幽灵是黑天魔神所著奇幻小说《废土》的最新章节，棋子小说提供无弹窗阅读！"><link href="http://www.qzxs.cc/xiaoshuo/rbd2vo1/chapter/32g5vo1.html" rel="canonical"><meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1, user-scalable=no, viewport-fit=cover"><meta name="renderer" content="webkit|ie-comp|ie-stand"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><meta http-equiv="Cache-Control" content="no-transform"><meta http-equiv="Cache-Control" content="no-siteapp"><meta name="applicable-device" content="pc,mobile"><link rel="stylesheet" type="text/css" href="/static/biquge2023/book.css?v=20231124"><script>var classVal="theme5";if(window.localStorage.getItem("bgcolor")){document.getElementsByTagName("html")[0].setAttribute("class",window.localStorage.g

In [5]:
doc = pq(filename='32g5vo1.html')

In [7]:
title = doc('.read .booktitle h1').text()
print(title)

第四百六七节 幽灵


In [8]:
content = doc('.read .content p').text()
print(content)

黑天魔神提示您：看后求收藏（棋子小说qzxs.cc），接着再看更方便。 走廊，非常幽深。 配合几分钟前刚刚离开的电梯，还有表示楼层的“一21，字样，即便是林翔，也有种从地面直接坠入地狱深处的错觉。 在一道六棱形钢制大门前，雷契尔将军停下脚步，从贴身衣袋里取出一张磁卡，用力刷过旁边墙壁上的电子感应器。随即，从棱形门壁的背后，传来齿轮转动的清晰声响。 “这里并不设防。” 将军深深地吸了口气，说：“你可能会觉得奇怪，为什么不适用保密级别更高的虹膜探测或者个人指纹密码？但你很快就能知道，这些所谓的屏障，在那个人面前根本没有丝毫效果。即便没有通行卡，你也很容易就能进入。我并不是刻意制止或者故意恐吓，那个人。”。”他已经超乎正常想象的范围。或许，只有你和他之间，才有真正能够谈论的话题。但是不管怎么样，我希望你能够记住之前说过的那些话。这个世界已经太过混乱，我不希望看到战争，有很多人。”。”不应该默默无闻的死去。” 林翔黑色的瞳孔焦点落在了雷契尔身上。刹那间，将军只觉得自己的身体从内到外都冒出森森寒意，似乎已变成完全透明。在这种无所遁形的目光下，所有的秘密都无从隐藏。 “你和我所见过的一些骷髅骑士不同。至少。”。”你仍然保持着某种信念。” 林翔平静地说：、‘向城外的军队请降吧！你并非战败，只是因为责任。” 。”。” 全封闭的棱形走庇，很宽敞。地面与头顶都装设着淡蓝色的冷光灯，黑暗被彻底驱逐，却丝毫感觉不到温暖。 尽头，是一道同样形状的感应门。不需要触摸，它已经自动分朝两边，让出足够通行的过道。走进其中，林翔却如同被电击一样牢牢定住，站在原地，久久未能移开脚步。 这是一个非常宽大的房间，面积至少超过两千平米。沿着墙壁，摆放着各种复杂的电子仪器，还有一张张堆放器材的工台。柔和的白色灯光从头顶洒落下来，足够看清楚其中的每一个角落。 一个身穿白色家居便服的青年，正对着林翔，坐在房间南角的椅子上，聚精会神看着一本平放在桌面上的书。 他的外表年龄最多只有二十岁。眼睛是欧裔人种特有的淡蓝色，长长的金色头发，在脑后用黑色缎带系成马尾。皮肤很白，似乎长时间没有接触过阳光。在这种苍白肤色的衬托下，嘴唇上的淡红也显得越发鲜艳。尤其是从林翔站立的角度望去，感觉。”。”就像是传说中刚刚饮过人血的吸血鬼。 目光与这名男子接触的一刹那，林翔忽然产生出一种极其微妙，也非常奇怪的感觉。 自己。”。”似乎认识他。 

In [9]:
with open('novle.txt', 'w', encoding='utf-8') as f:
    f.write(content)

In [2]:
doc = pq(filename='book_detail.html')

In [4]:
doc('.comments').text()

'你会重新爱这个曾是你自己的陌生人。\n喜欢他以诗意和文学的形式解读武侠小说。他会用塞林格的短篇串联程灵素的悲剧，也会用博尔赫斯的哲学观来形容古龙笔下的英雄，还以萨特和萨冈来形容自己的所感所悟。他的文字充满才华，诗句也穿插妙用，很像以前读书时会仰慕的文科才子。\n大学时读过，现在重读，缅怀大学岁月。2020年读的第一本书。\n江湖中人\n品读武侠，北大才子读武侠和我们普通人就是不一样，我们看剧情热闹，人家看的是人生人事\n多年前听闻此书，一直没买到。作者对古金的人物，兵器，每个人的个性真的是如数家珍，借由这些引出自己的人生观及道路走向。书中看到很多北大才子们的诗作，对青春江湖的缅怀，是另一得。\n除却斯人海子的另外一个北大才子江湖。'

In [18]:
text = '''
                喜欢他以诗意和文学的形式解读武侠小说。他会用塞林格的短篇串联程灵素的悲剧，也会用博尔赫斯的哲学观来形容古龙笔下的英雄，还以萨特和萨冈来形容自己的所感所悟。他的文字充满才华，诗句也穿插妙用，很像以前读书时会仰慕的文科才子。
              '''

In [19]:
text

'\n                喜欢他以诗意和文学的形式解读武侠小说。他会用塞林格的短篇串联程灵素的悲剧，也会用博尔赫斯的哲学观来形容古龙笔下的英雄，还以萨特和萨冈来形容自己的所感所悟。他的文字充满才华，诗句也穿插妙用，很像以前读书时会仰慕的文科才子。\n              '

In [12]:
results = re.search(r'[^\x00-\xff]', text).group()

In [10]:
for item in results.groups():
    print(item)

AttributeError: 'str' object has no attribute 'groups'

In [20]:
re.search(r'[^\x00-\xff].*', text).group()

'喜欢他以诗意和文学的形式解读武侠小说。他会用塞林格的短篇串联程灵素的悲剧，也会用博尔赫斯的哲学观来形容古龙笔下的英雄，还以萨特和萨冈来形容自己的所感所悟。他的文字充满才华，诗句也穿插妙用，很像以前读书时会仰慕的文科才子。'